In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
from glob import glob

import warnings
warnings.filterwarnings('ignore')

In [2]:
fold = 1

In [3]:
train_path = "../input/anomaly-detection/open/open/train/train/"
test_path = "../input/anomaly-detection/open/open/test/test/"

In [4]:
train = pd.read_csv("../input/anomaly-detection/open/open/train_df.csv")
test = pd.read_csv("../input/anomaly-detection/open/open/test_df.csv")
sub = pd.read_csv("../input/anomaly-detection/open/open/sample_submission.csv")
train['file_path'] = train_path + train['file_name']
test['file_path'] = test_path + test['file_name']
train.head()

,index,file_name,class,state,label,file_path
0,0,10000.png,transistor,good,transistor-good,../input/anomaly-detection/open/open/train/tra...
1,1,10001.png,capsule,good,capsule-good,../input/anomaly-detection/open/open/train/tra...
2,2,10002.png,transistor,good,transistor-good,../input/anomaly-detection/open/open/train/tra...
3,3,10003.png,wood,good,wood-good,../input/anomaly-detection/open/open/train/tra...
4,4,10004.png,bottle,good,bottle-good,../input/anomaly-detection/open/open/train/tra...


In [5]:
train_labels = train["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [6]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.5/431.5 KB 2.1 MB/s eta 0:00:00a 0:00:01


In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import timm

# torch.manual_seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
from albumentations.pytorch.transforms import ToTensorV2
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

In [9]:
def get_train_augmentation(img_size, ver):
    if ver==1: # for valid, test
        transform = transforms.Compose([
                transforms.Resize((img_size, img_size)),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                ])

    if ver==2:
        transform = transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomAffine((-20, 20)),
                transforms.RandomRotation(90),
                transforms.Resize((img_size, img_size)),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
            ])
    if ver==3:
        transform = transforms.Compose([
                transforms.Resize((img_size+28, img_size+28)),
                transforms.RandomCrop(img_size),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomAffine((-20, 20)),
                transforms.RandomRotation(90),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
            ])
    if ver==4:
        transform = transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
#                 transforms.RandomApply([transforms.RandomAffine((-20, 20), translate=(0.1, 0.1))], p=0.3),
                transforms.RandomAffine((-20, 20), translate=(0.1, 0.1)),
                transforms.RandomRotation(90),
                transforms.Resize((img_size, img_size)),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
            ])
    if ver==5:
        transform = transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomApply([transforms.RandomAffine(0, translate=(0.1, 0.1), scale=(1, 1.2))], p=0.5),
                transforms.RandomApply([transforms.RandomRotation(90)], p=0.5),
                transforms.Resize((img_size, img_size)),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
            ]) 
    if ver==6:
        transform = Compose([
                  Transpose(p=0.5),
                  HorizontalFlip(p=0.5),
                  VerticalFlip(p=0.5),
                  ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=45, interpolation=0, border_mode=0, p=0.5),
                  RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
                  Resize(img_size, img_size),
                  Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
                  ToTensorV2(p=1.0),
                  ], p=1.)
    if ver==7:
        transform = Compose([
                  Resize(img_size, img_size),
                  Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
                  ToTensorV2(p=1.0),
                  ], p=1.)    
        
    return transform      

class Trainset(Dataset):
    def __init__(self, df, labels, transform=None):
        self.img_paths = df['file_path'].values
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
#         image = cv2.imread(self.img_paths[idx]).astype(np.float32)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0

#         if self.transform is not None:
#             image = self.transform(torch.from_numpy(image.transpose(2,0,1)))

        image = cv2.imread(self.img_paths[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.uint8) 
        if self.transform is not None:
            image = self.transform(image=image)['image'] 
        label = self.labels[idx]
        return image, label

class Testset(Dataset):
    def __init__(self, df, transform=None):
        self.img_paths = df['file_path'].values
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
#         image = cv2.imread(self.img_paths[idx]).astype(np.float32)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0

#         if self.transform is not None:
#             image = self.transform(torch.from_numpy(image.transpose(2,0,1)))

        image = cv2.imread(self.img_paths[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.uint8) 
        if self.transform is not None:
            image = self.transform(image=image)['image'] 

        return image

class Net(nn.Module):
    def __init__(self, name):
        super(Net, self).__init__()
        self.cnn = timm.create_model( # timm ImageNet pre-trained 모델 load
            name,
            pretrained=True,
            num_classes = 88
        )
        
    def forward(self, x):
        out = self.cnn(x)
        return out

In [10]:
for i in timm.list_models():
    if "rexnet" in i:
        print(i)

rexnet_100
rexnet_130
rexnet_150
rexnet_200
rexnetr_100
rexnetr_130
rexnetr_150
rexnetr_200


In [11]:
model = Net('rexnet_200').to(device)   #best
# model = Net('regnety_040').to(device)
# model = Net('efficientnet_b8').to(device)
# model = Net('regnety_320').to(device)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rexnet/rexnetv1_200-8c0b7f2d.pth" to /root/.cache/torch/hub/checkpoints/rexnetv1_200-8c0b7f2d.pth


In [12]:
train_transform = get_train_augmentation(img_size=512, ver=6)
test_transform = get_train_augmentation(img_size=512, ver=7)

In [13]:
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=41)
for f, (train_idx, val_idx) in enumerate(kf.split(range(len(train)), y=np.array(train_labels))):
    train.loc[val_idx, 'fold'] = f

In [14]:
df_val = train[train['fold'] == fold].reset_index(drop=True)
df_train = train[train['fold'] != fold].reset_index(drop=True)

train_labels = df_train["label"]
train_labels = [label_unique[k] for k in train_labels]

val_labels = df_val["label"]
val_labels = [label_unique[k] for k in val_labels]

In [15]:
batch_size = 16

# Train
train_dataset = Trainset(df_train, np.array(train_labels), train_transform)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, num_workers=4)

# Val
val_dataset = Trainset(df_val, np.array(val_labels), test_transform)
val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size, num_workers=4)

# Test
test_dataset = Testset(test, test_transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size, num_workers=4)

In [16]:
# batch_size = 16

# # Train
# train_dataset = Trainset(train, np.array(train_labels), train_transform)
# train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, num_workers=4)

# # Test
# test_dataset = Testset(test, test_transform)
# test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size, num_workers=4)

In [17]:
# # train_dataset = Trainset(train, np.array(train_labels), train_transform)
# # train_loader = DataLoader(train_dataset, shuffle=True, batch_size=32, num_workers=4)

# for i in train_loader:
#     s = 1
#     plt.figure(figsize=(20,20))
#     for img in i[0]:
#         plt.subplot(int(len(i[0])/4), 4, s)
#         plt.imshow(np.transpose(img, (1,2,0)))
#         s = s + 1
#     break

In [18]:
# for i in test_loader:
#     s = 1
#     plt.figure(figsize=(16,10))
#     for img in i:
#         plt.subplot(int(len(i)/4), 4, s)
#         plt.imshow(np.transpose(img, (1,2,0)))
#         s = s + 1
#     break

In [19]:
epochs = 70
warm_epoch = 5
score = 0

def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-6, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-3, 
                                               steps_per_epoch=len(train_loader), 
                                                epochs=epochs)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler()

best=0
for epoch in range(epochs):
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in tqdm((train_loader)):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
#         if epoch <= warm_epoch:
#             warmup_scheduler.step()
        model.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        if epoch > warm_epoch:
            scheduler.step()
                                               
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
        
    train_f1 = score_function(train_y, train_pred)
    
    model.eval()
    val_pred = []
    val_y = []
    with torch.no_grad():
        for batch in tqdm((val_loader)):
            x = torch.tensor(batch[0], dtype=torch.float32, device=device)
            y = torch.tensor(batch[1], dtype=torch.long, device=device)
            with torch.cuda.amp.autocast():
                pred = model(x)
            val_pred += pred.argmax(1).detach().cpu().numpy().tolist()
            val_y += y.detach().cpu().numpy().tolist()
    val_f1 = score_function(val_y, val_pred)
    
    print("Epoch: {}, loss: {}, f1score: {}, val_score: {}".format(epoch+1, train_loss, train_f1, val_f1))
    if score < val_f1:
        score = val_f1
        torch.save(model.state_dict(), "./bestmodel{}.pth".format(fold))

100%|██████████| 54/54 [00:30<00:00,  1.78it/s]


Epoch: 1, loss: 1.451633489020516, f1score: 0.1481999434207724, val_score: 0.16785636064661547


100%|██████████| 54/54 [00:29<00:00,  1.86it/s]


Epoch: 2, loss: 0.7959908458674065, f1score: 0.18423385764586217, val_score: 0.19977959558321148


100%|██████████| 54/54 [00:29<00:00,  1.85it/s]


...

Epoch: 68, loss: 0.002676007068045787, f1score: 0.9993887779634082, val_score: 0.8346287102702331


100%|██████████| 54/54 [00:29<00:00,  1.83it/s]


Epoch: 69, loss: 0.005220673073117976, f1score: 0.9948208052951807, val_score: 0.8151126104280464


100%|██████████| 54/54 [00:29<00:00,  1.85it/s]

Epoch: 70, loss: 0.005262799630655306, f1score: 0.9954505589490753, val_score: 0.830713150174898


In [20]:
# epochs = 50
# warm_epoch = 5
# score = 0

# def score_function(real, pred):
#     score = f1_score(real, pred, average="macro")
#     return score

# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-3)
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.998)
# criterion = nn.CrossEntropyLoss()
# scaler = torch.cuda.amp.GradScaler()

# best=0
# for epoch in range(epochs):
#     train_loss = 0
#     train_pred=[]
#     train_y=[]
#     model.train()
#     for batch in tqdm((train_loader)):
#         optimizer.zero_grad()
#         x = torch.tensor(batch[0], dtype=torch.float32, device=device)
#         y = torch.tensor(batch[1], dtype=torch.long, device=device)
# #         if epoch <= warm_epoch:
# #             warmup_scheduler.step()
#         model.zero_grad(set_to_none=True)
#         with torch.cuda.amp.autocast():
#             pred = model(x)
#         loss = criterion(pred, y)


#         scaler.scale(loss).backward()
#         scaler.step(optimizer)
#         scaler.update()
#         # Clip gradient
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 15.0)
        
#         if epoch > warm_epoch:
#             scheduler.step()
                                               
#         train_loss += loss.item()/len(train_loader)
#         train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
#         train_y += y.detach().cpu().numpy().tolist()
        
#     train_f1 = score_function(train_y, train_pred)
#     print("Epoch: {}, f1score: {}, loss: {}".format(epoch+1, train_f1, train_loss))
#     if score < train_f1:
#         score = train_f1
#         torch.save(model.state_dict(), "./model.pth")

In [21]:
torch.save(model.state_dict(), "./model{}.pth".format(fold))

In [22]:
# model = Net('rexnet_200').to(device)
# model.load_state_dict(torch.load('../input/anomaly-detection/bestmodel3.pth'))

In [23]:
model.eval()
f_pred = []
preds_list = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        x = torch.tensor(batch, dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model(x)
            pred = torch.softmax(pred, dim=1)
            preds_list.extend(pred.cpu().tolist())
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

100%|██████████| 135/135 [01:10<00:00,  1.90it/s]


In [24]:
preds = pd.DataFrame(preds_list)
preds.to_csv('./preds{}.csv'.format(fold), index = False)
preds.head()

,0,1,2,3,4,5,6,7,8,9,...,78,79,80,81,82,83,84,85,86,87
0,5.547519e-10,1.549312e-11,4.249913e-09,1.528046e-12,3.045278e-11,3.910215e-11,3.695445e-10,1.033928e-12,6.751665e-09,2.168201e-10,...,4.860668e-09,4.811082e-09,1.883766e-10,5.251501e-11,5.655245e-12,4.790894e-11,1.977438e-12,7.220134e-11,1.113584e-12,1.174059e-11
1,1.785198e-10,1.237409e-12,1.461093e-11,2.239141e-10,3.176250e-10,4.810927e-09,1.248096e-10,7.193901e-09,1.811812e-09,2.241998e-08,...,4.587008e-11,3.432157e-11,1.552059e-12,1.843391e-11,2.433302e-09,4.337830e-09,3.896008e-09,2.258570e-11,5.808439e-11,2.159356e-11
2,1.694950e-11,1.977436e-12,3.063170e-11,5.460690e-11,2.917194e-11,2.650557e-12,6.369578e-10,1.227818e-12,1.254669e-11,7.639739e-12,...,3.364234e-11,3.964036e-11,3.723312e-12,1.371850e-09,5.198907e-13,7.276752e-11,4.177440e-13,6.663499e-12,4.643488e-11,8.260527e-12
3,4.368728e-14,9.898502e-17,1.739317e-08,8.486990e-14,2.558230e-14,1.456765e-18,9.198683e-11,1.622695e-15,5.183702e-13,3.781383e-13,...,5.607883e-16,2.658550e-18,4.685562e-16,4.472329e-14,3.369807e-13,5.810276e-14,1.244163e-15,8.915670e-11,6.016380e-16,2.513279e-15
4,4.326711e-10,3.641678e-10,8.507329e-11,7.665536e-09,2.408994e-11,1.657384e-09,1.639275e-09,1.090231e-10,2.524981e-10,2.529917e-10,...,2.507779e-10,5.904375e-11,1.297213e-10,8.114999e-10,1.727619e-09,7.429692e-09,1.502530e-07,5.880479e-12,6.004112e-10,1.528490e-10


In [25]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

In [26]:
sub["label"] = f_result
sub.to_csv('./sub.csv', index = False)
sub.head()

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-good
3,3,tile-gray_stroke
4,4,tile-good
